In [ ]:
import numpy as np
import os

In [ ]:
mainpath   = '/home/taylor/Documents/Research/spitzer/MEGA/'
planets = ['CoRoT-2b', 'HAT-P-7b', 'KELT-16b', 'KELT-9b', 'MASCARA-1b', 'Qatar1b', 'WASP-14b', 'WASP-18b', 'WASP-19b', 'WASP-33b', 'WASP-43b', 'WASP-12b']
channels = ['ch2', 'ch2', 'ch2', 'ch2', 'ch2', 'ch2', 'ch2', 'ch2', 'ch2', 'ch2', 'ch2', 'ch2', 'ch2']
twoLimits = False
bicThresh = 5

In [ ]:
def roundToSigFigs(x, sigFigs=2):
    nDec = -int(np.floor(np.log10(np.abs(x))))+sigFigs-1
    rounded = np.round(x, nDec)
    if nDec <= 0:
        #remove decimals if this should be an integer
        rounded = int(rounded)
    output = str(rounded)
    if nDec > 1 and np.rint(rounded*10**nDec) % 10 == 0:
        #take care of the trailing zero issue
        output += '0'
    return nDec, output
def roundToDec(x, nDec=2):
    rounded = np.round(x, nDec)
    if nDec <= 0:
        #remove decimals if this should be an integer
        rounded = int(rounded)
    output = str(rounded)
    if nDec > 1 and np.rint(rounded*10**nDec) % 10 == 0:
        #take care of the trailing zero issue
        output += '0'
    return output

In [ ]:
output1 = ''
output2 = ''
output3 = ''
output4 = ''
for iterNum in range(len(planets)):
    channel = channels[iterNum]
    planet = planets[iterNum]
    
    output1 += planet+' '+channel+'\n'
    output2 += planet+' '+channel+'\n'
    
    with open(mainpath+planet+'/analysis/'+channel+'/bestPhOption.txt') as f:
        line = f.readline().strip()
        foldername = mainpath+line[line.find(planet):]+'/'

    pldFoldernames = np.array(['/'.join(foldername.split('/')[:-2])+'/'+np.array(x)[0].split('/')[-1]+'/' for x in os.walk('/'.join(foldername.split('/')[:-2])+'/') if '4umPLD' in x[0].split('/')[-1] or '3umPLD' in x[0].split('/')[-1]])

    modes = np.sort([x[0].split('/')[-1] for x in os.walk(foldername)][1:])
    for i in range(len(pldFoldernames)):
        modes = np.append(modes, np.sort([x[0].split('/')[-1] for x in os.walk(pldFoldernames[i])][1:]))

    #find the best modes
    BICs = []
    for mode in modes:
        if 'PLD' in mode:
            if channel=='ch2':
                folder='4um'
            else:
                folder='3um'
            if '3x3' in mode:
                folder += 'PLD_3x3/'
            else:
                folder += 'PLD_5x5/'
            savepath   = '/'.join(foldername.split('/')[:-2])+'/'+folder + mode + '/'+ 'ResultMCMC_'+mode+'_Params.npy'
        else:
            savepath   = foldername + mode + '/'+ 'ResultMCMC_'+mode+'_Params.npy'
            
        if not os.path.exists(savepath):
            BICs.append(np.inf)
        else:
            ResultMCMC = np.load(savepath)
            BICs.append(-2*ResultMCMC['evidenceB'][0])
    BICs = np.array(BICs)

    try:
        bestBIC = np.min(BICs)
    except:
        # There are no analyses for this planet
        print('No analyses for planet', planet, 'channel', channel)

    ######################## MAKE TABLE2 #1 AND #2 ########################

    for i in range(len(modes)):

        mode = modes[i]

        if 'PLD' in mode:
            if channel=='ch2':
                folder='4um'
            else:
                folder='3um'
            if '3x3' in mode:
                folder += 'PLD_3x3/'
            else:
                folder += 'PLD_5x5/'
            savepath   = '/'.join(foldername.split('/')[:-2])+'/'+folder + mode + '/'+ 'ResultMCMC_'+mode+'_Params.npy'
        else:
            savepath   = foldername + mode + '/'+ 'ResultMCMC_'+mode+'_Params.npy'
        
        if not os.path.exists(savepath):
            continue

        if int(np.round(BICs[i]-bestBIC, 0)) <= bicThresh:
            output1 += '\\textbf{'
            output2 += '\\textbf{'
        else:
            output1 += '        '
            output2 += '        '

        ResultMCMC = np.load(savepath)

        if 'poly' in mode.lower():
            ind = mode.lower().find('poly')
            output1 += mode[ind:ind+5]
            output2 += mode[ind:ind+5]
        elif 'bliss' in mode.lower():
            output1 += 'BLISS'
            output2 += 'BLISS'
        elif 'pldaper1' in mode.lower():
            output1 += 'PLDAper1'
            output2 += 'PLDAper1'
        elif 'pldaper2' in mode.lower():
            output1 += 'PLDAper2'
            output2 += 'PLDAper2'
        elif 'pld1' in mode.lower():
            output1 += 'PLD1'
            output2 += 'PLD1'
        elif 'pld2' in mode.lower():
            output1 += 'PLD2'
            output2 += 'PLD2'
        elif 'gp' in mode.lower():
            output1 += 'GP'
            output2 += 'GP'

        if 'pld' in mode.lower():
            if '3x3' in mode.lower():
                output1 += ' 3x3'
                output2 += ' 3x3'
            elif '5x5' in mode.lower():
                output1 += ' 5x5'
                output2 += ' 5x5'
            
        if 'tslope' in mode.lower():
            output1 += '*$f$(t)'
            output2 += '*$f$(t)'

        if 'hside' in mode.lower():
            output1 += '*Step'
            output2 += '*Step'

        if 'psfw' in mode.lower():
            output1 += ', PSFW'
            output2 += ', PSFW'
            
        if int(np.round(BICs[i]-bestBIC, 0)) <= bicThresh:
            output1 += '}& '
            output2 += '}& '
        else:
            output1 += ' & '
            output2 += ' & '

        ind = mode.lower().find('_v')
        vorder = mode[ind+1:].split('_')[0][1:]

        if vorder == '1':
            output1 += '1st Order'
            output2 += '1st Order'
        elif vorder == '2':
            output1 += '2nd Order'
            output2 += '2nd Order'

        if 'ellipse' in mode.lower() and 'offset' in mode.lower():
            output1 += ' + Rotated Ellipse & '
            output2 += ' + Rotated Ellipse & '
        elif 'ellipse' in mode.lower():
            output1 += ' + Ellipse & '
            output2 += ' + Ellipse & '
        else:
            output1 += ' & '
            output2 += ' & '

        nData = ResultMCMC['chi2B'][0]/ResultMCMC['chi2datum'][0]
            
        val = ResultMCMC['A']
        if twoLimits:
            nDec1, err1 = roundToSigFigs(val[1])
            nDec2, err2 = roundToSigFigs(val[2])
            nDec = np.max([nDec1, nDec2])
            val = roundToDec(val[0], nDec)
            output1 += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
        else:
            nDec, err = roundToSigFigs(np.mean(val[1:]))
            val = roundToDec(val[0], nDec)
            output1 += '$'+val+'\pm'+err+'$ & '


        val = ResultMCMC['B']
        if twoLimits:
            nDec1, err1 = roundToSigFigs(val[1])
            nDec2, err2 = roundToSigFigs(val[2])
            nDec = np.max([nDec1, nDec2])
            val = roundToDec(val[0], nDec)
            output1 += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
        else:
            nDec, err = roundToSigFigs(np.mean(val[1:]))
            val = roundToDec(val[0], nDec)
            output1 += '$'+val+'\pm'+err+'$ & '

        if vorder == '1':
            output1 += '. & . & '
        else:
            val = ResultMCMC['C']
            if twoLimits:
                nDec1, err1 = roundToSigFigs(val[1])
                nDec2, err2 = roundToSigFigs(val[2])
                nDec = np.max([nDec1, nDec2])
                val = roundToDec(val[0], nDec)
                output1 += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
            else:
                nDec, err = roundToSigFigs(np.mean(val[1:]))
                val = roundToDec(val[0], nDec)
                output1 += '$'+val+'\pm'+err+'$ & '

            val = ResultMCMC['D']
            if twoLimits:
                nDec1, err1 = roundToSigFigs(val[1])
                nDec2, err2 = roundToSigFigs(val[2])
                nDec = np.max([nDec1, nDec2])
                val = roundToDec(val[0], nDec)
                output1 += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
            else:
                nDec, err = roundToSigFigs(np.mean(val[1:]))
                val = roundToDec(val[0], nDec)
                output1 += '$'+val+'\pm'+err+'$ & '

        val = ResultMCMC['fp']*1e6
        if twoLimits:
            nDec1, err1 = roundToSigFigs(val[1])
            nDec2, err2 = roundToSigFigs(val[2])
            nDec = np.max([nDec1, nDec2])
            val = roundToDec(val[0], nDec)
            output1 += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
        else:
            nDec, err = roundToSigFigs(np.mean(val[1:]))
            val = roundToDec(val[0], nDec)
            output1 += '$'+val+'\pm'+err+'$ & '

        val = ResultMCMC['sigF']*1e6
        if twoLimits:
            nDec1, err1 = roundToSigFigs(val[1])
            nDec2, err2 = roundToSigFigs(val[2])
            nDec = np.max([nDec1, nDec2])
            val = roundToDec(val[0], nDec)
            output1 += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
        else:
            nDec, err = roundToSigFigs(np.mean(val[1:]))
            val = roundToDec(val[0], nDec)
            output1 += '$'+val+'\pm'+err+'$ & '

        output1 += str(np.round(ResultMCMC['chi2B'][0], 2))+' & '







        val = ResultMCMC['rp']
        if twoLimits:
            nDec1, err1 = roundToSigFigs(val[1])
            nDec2, err2 = roundToSigFigs(val[2])
            nDec = np.max([nDec1, nDec2])
            val = roundToDec(val[0], nDec)
            output2 += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
        else:
            nDec, err = roundToSigFigs(np.mean(val[1:]))
            val = roundToDec(val[0], nDec)
            output2 += '$'+val+'\pm'+err+'$ & '

#         if 'ellipse' in mode.lower():
#             val = ResultMCMC['r2']
#             if twoLimits:
#                 nDec1, err1 = roundToSigFigs(val[1])
#                 nDec2, err2 = roundToSigFigs(val[2])
#                 nDec = np.max([nDec1, nDec2])
#                 val = roundToDec(val[0], nDec)
#                 output2 += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
#             else:
#                 nDec, err = roundToSigFigs(np.mean(val[1:]))
#                 val = roundToDec(val[0], nDec)
#                 output2 += '$'+val+'\pm'+err+'$ & '

#             val = [0,0,0]
#             r2 = ResultMCMC['r2'][0]
#             rp = ResultMCMC['rp'][0]
#             val[0] = r2/rp
#             r2err = np.max([ResultMCMC['r2'][1], ResultMCMC['r2'][2]])
#             rperr = np.max([ResultMCMC['rp'][1], ResultMCMC['rp'][2]])
#             val[1] = np.sqrt((r2err/rp)**2+(r2/rp**2*rperr)**2)
#             nDec, err = roundToSigFigs(val[1])
#             val = roundToDec(val[0], nDec)
#             output2 += '$'+val+'\pm'+err+'$ & '
#         else:
#             output2 += '. & . & '

        val = ResultMCMC['offset']
        if twoLimits:
            nDec1, err1 = roundToSigFigs(val[1])
            nDec2, err2 = roundToSigFigs(val[2])
            nDec = np.max([nDec1, nDec2])
            val = roundToDec(-val[0], nDec)
            output2 += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
        else:
            nDec, err = roundToSigFigs(np.mean(val[1:]))
            val = roundToDec(-val[0], nDec)
            output2 += '$'+val+'\pm'+err+'$ & '

        if 'v2' in mode.lower():
            Cs =  ResultMCMC['C'][0]
            Cs_err = np.mean(ResultMCMC['C'][1:])
            Ds =  ResultMCMC['D'][0]
            Ds_err = np.mean(ResultMCMC['D'][1:])
            offset2 = 180-np.arctan2(Ds, Cs)*180/np.pi/2
            offset2_err = 1/(1+(Ds/Cs)**2)*np.sqrt((Ds_err/Cs)**2+(Ds/Cs**2*Cs_err)**2)*180/np.pi/2
            nDec, err = roundToSigFigs(offset2_err)
            val = roundToDec(offset2, nDec)
            output2 += '$'+val+'\pm'+err+'$ & '
        else:
            output2 += ' & '

        val = ResultMCMC['tDay']
        if twoLimits:
            nDec1, err1 = roundToSigFigs(val[1])
            nDec2, err2 = roundToSigFigs(val[2])
            nDec = np.max([nDec1, nDec2])
            val = roundToDec(val[0], nDec)
            output2 += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
        else:
            nDec, err = roundToSigFigs(np.mean(val[1:]))
            val = roundToDec(val[0], nDec)
            output2 += '$'+val+'\pm'+err+'$ & '

        val = ResultMCMC['tNight']
        if twoLimits:
            nDec1, err1 = roundToSigFigs(val[1])
            nDec2, err2 = roundToSigFigs(val[2])
            nDec = np.max([nDec1, nDec2])
            val = roundToDec(val[0], nDec)
            output2 += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
        else:
            nDec, err = roundToSigFigs(np.mean(val[1:]))
            val = roundToDec(val[0], nDec)
            output2 += '$'+val+'\pm'+err+'$ & '



        if int(np.round(BICs[i]-bestBIC, 0)) <= bicThresh:
            output1 += '\\textbf{'
            output2 += '\\textbf{'
        output1 += str(int(np.round(BICs[i]-bestBIC, 0)))
        output2 += str(int(np.round(BICs[i]-bestBIC, 0)))
        if int(np.round(BICs[i]-bestBIC, 0)) <= bicThresh:
            output1 += '}'
            output2 += '}'

        output1 += ' \\\\ % \n'
        output2 += ' \\\\ % \n'


        ######################## MAKE TABLE #3 and 4 ########################

        if BICs[i] != bestBIC:
            continue

        output3 += planet+' '+channel+' & '
        output4 += planet+' '+channel+' & '

        if 'poly' in mode.lower():
            ind = mode.lower().find('poly')
            output3 += mode[ind:ind+5]
            output4 += mode[ind:ind+5]
        elif 'bliss' in mode.lower():
            output3 += 'BLISS'
            output4 += 'BLISS'
        elif 'pldaper1' in mode.lower():
            output3 += 'PLDAper1'
            output4 += 'PLDAper1'
        elif 'pldaper2' in mode.lower():
            output3 += 'PLDAper2'
            output4 += 'PLDAper2'
        elif 'pld1' in mode.lower():
            output3 += 'PLD1'
            output4 += 'PLD1'
        elif 'pld2' in mode.lower():
            output3 += 'PLD2'
            output4 += 'PLD2'
        elif 'gp' in mode.lower():
            output3 += 'GP'
            output4 += 'GP'

        if 'pld' in mode.lower():
            if '3x3' in mode.lower():
                output3 += ' 3x3'
                output4 += ' 3x3'
            elif '5x5' in mode.lower():
                output3 += ' 5x5'
                output4 += ' 5x5'
                
        if 'tslope' in mode.lower():
            output3 += '*$f$(t)'
            output4 += '*$f$(t)'

        if 'hside' in mode.lower():
            output3 += '*Step'
            output4 += '*Step'

        if 'psfw' in mode.lower():
            output3 += ', PSFW'
            output4 += ', PSFW'
            
        output3 += ', '
        output4 += ', '

        ind = mode.lower().find('_v')
        vorder = mode[ind+1:].split('_')[0][1:]

        if vorder == '1':
            output3 += '1st Order'
            output4 += '1st Order'
        elif vorder == '2':
            output3 += '2nd Order'
            output4 += '2nd Order'

        if 'ellipse' in mode.lower() and 'offset' in mode.lower():
            output3 += ' + Rotated Ellipse & '
            output4 += ' + Rotated Ellipse & '
        elif 'ellipse' in mode.lower():
            output3 += ' + Ellipse & '
            output4 += ' + Ellipse & '
        else:
            output3 += ' & '
            output4 += ' & '

        val = ResultMCMC['A']
        if twoLimits:
            nDec1, err1 = roundToSigFigs(val[1])
            nDec2, err2 = roundToSigFigs(val[2])
            nDec = np.max([nDec1, nDec2])
            val = roundToDec(val[0], nDec)
            output3 += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
        else:
            nDec, err = roundToSigFigs(np.mean(val[1:]))
            val = roundToDec(val[0], nDec)
            output3 += '$'+val+'\pm'+err+'$ & '

        val = ResultMCMC['B']
        if twoLimits:
            nDec1, err1 = roundToSigFigs(val[1])
            nDec2, err2 = roundToSigFigs(val[2])
            nDec = np.max([nDec1, nDec2])
            val = roundToDec(val[0], nDec)
            output3 += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
        else:
            nDec, err = roundToSigFigs(np.mean(val[1:]))
            val = roundToDec(val[0], nDec)
            output3 += '$'+val+'\pm'+err+'$ & '

        if vorder == '1':
            output3 += '. & . & '
        else:
            val = ResultMCMC['C']
            if twoLimits:
                nDec1, err1 = roundToSigFigs(val[1])
                nDec2, err2 = roundToSigFigs(val[2])
                nDec = np.max([nDec1, nDec2])
                val = roundToDec(val[0], nDec)
                output3 += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
            else:
                nDec, err = roundToSigFigs(np.mean(val[1:]))
                val = roundToDec(val[0], nDec)
                output3 += '$'+val+'\pm'+err+'$ & '

            val = ResultMCMC['D']
            if twoLimits:
                nDec1, err1 = roundToSigFigs(val[1])
                nDec2, err2 = roundToSigFigs(val[2])
                nDec = np.max([nDec1, nDec2])
                val = roundToDec(val[0], nDec)
                output3 += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
            else:
                nDec, err = roundToSigFigs(np.mean(val[1:]))
                val = roundToDec(val[0], nDec)
                output3 += '$'+val+'\pm'+err+'$ & '

        val = ResultMCMC['fp']*1e6
        if twoLimits:
            nDec1, err1 = roundToSigFigs(val[1])
            nDec2, err2 = roundToSigFigs(val[2])
            nDec = np.max([nDec1, nDec2])
            val = roundToDec(val[0], nDec)
            output3 += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
        else:
            nDec, err = roundToSigFigs(np.mean(val[1:]))
            val = roundToDec(val[0], nDec)
            output3 += '$'+val+'\pm'+err+'$ & '






        val = ResultMCMC['rp']
        if twoLimits:
            nDec1, err1 = roundToSigFigs(val[1])
            nDec2, err2 = roundToSigFigs(val[2])
            nDec = np.max([nDec1, nDec2])
            val = roundToDec(val[0], nDec)
            output4 += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
        else:
            nDec, err = roundToSigFigs(np.mean(val[1:]))
            val = roundToDec(val[0], nDec)
            output4 += '$'+val+'\pm'+err+'$ & '

        val = ResultMCMC['offset']
        if twoLimits:
            nDec1, err1 = roundToSigFigs(val[1])
            nDec2, err2 = roundToSigFigs(val[2])
            nDec = np.max([nDec1, nDec2])
            val = roundToDec(-val[0], nDec)
            output4 += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
        else:
            nDec, err = roundToSigFigs(np.mean(val[1:]))
            val = roundToDec(-val[0], nDec)
            output4 += '$'+val+'\pm'+err+'$ & '


        if 'v2' in mode.lower():
            Cs =  ResultMCMC['C'][0]
            Cs_err = np.mean(ResultMCMC['C'][1:])
            Ds =  ResultMCMC['D'][0]
            Ds_err = np.mean(ResultMCMC['D'][1:])
            offset2 = 180-np.arctan2(Ds, Cs)*180/np.pi/2
            offset2_err = 1/(1+(Ds/Cs)**2)*np.sqrt((Ds_err/Cs)**2+(Ds/Cs**2*Cs_err)**2)*180/np.pi/2
            nDec, err = roundToSigFigs(offset2_err)
            val = roundToDec(offset2, nDec)
            output4 += '$'+val+'\pm'+err+'$ & '
        else:
            output4 += ' & '

        val = ResultMCMC['tDay']
        if twoLimits:
            nDec1, err1 = roundToSigFigs(val[1])
            nDec2, err2 = roundToSigFigs(val[2])
            nDec = np.max([nDec1, nDec2])
            val = roundToDec(val[0], nDec)
            output4 += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
        else:
            nDec, err = roundToSigFigs(np.mean(val[1:]))
            val = roundToDec(val[0], nDec)
            output4 += '$'+val+'\pm'+err+'$ & '

        val = ResultMCMC['tNight']
        if twoLimits:
            nDec1, err1 = roundToSigFigs(val[1])
            nDec2, err2 = roundToSigFigs(val[2])
            nDec = np.max([nDec1, nDec2])
            val = roundToDec(val[0], nDec)
            output4 += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
        else:
            nDec, err = roundToSigFigs(np.mean(val[1:]))
            val = roundToDec(val[0], nDec)
            output4 += '$'+val+'\pm'+err+'$ & '


        output3 += str(np.round(ResultMCMC['logLB'][0]/nData, 2))
        output4 += str(np.round(ResultMCMC['logLB'][0]/nData, 2))

        output3 += ' \\\\ % \n'
        output4 += ' \\\\ % \n'

    output1 += '\n\n\n'
    output2 += '\n\n\n'
        

print(output1)

print('\n%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

print(output2)

print('\n%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

print(output3)

print('\n%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

print(output4)


In [ ]:
mode = 'PLDAper1_3x3_v1'
mode[mode.lower().find('pldaper'):].split('_')[0]